In [1]:
from pyspark.sql.functions import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = spark.read.csv("gs://msca-bdp-student-gcs/Group7_Final_Project/airline_reviews/AirlineReviews.csv", 
                                     header = True, inferSchema = True)

In [3]:
df.count()

130489

In [4]:
df.printSchema()

root
 |-- Aircraft: string (nullable = true)
 |-- AirlineName: string (nullable = true)
 |-- CabinType: string (nullable = true)
 |-- DateFlown: string (nullable = true)
 |-- DatePub: string (nullable = true)
 |-- EntertainmentRating: string (nullable = true)
 |-- FoodRating: string (nullable = true)
 |-- GroundServiceRating: string (nullable = true)
 |-- OriginCountry: string (nullable = true)
 |-- OverallScore: string (nullable = true)
 |-- Recommended: string (nullable = true)
 |-- Review: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- SeatComfortRating: string (nullable = true)
 |-- ServiceRating: string (nullable = true)
 |-- Slug: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- TravelType: string (nullable = true)
 |-- TripVerified: string (nullable = true)
 |-- ValueRating: string (nullable = true)
 |-- WifiRating: string (nullable = true)
 |-- unique_id: string (nullable = true)



In [5]:
df = df.withColumnRenamed('Aircraft', 'aircraft')
df = df.withColumnRenamed('AirlineName', 'airline_name')
df = df.withColumnRenamed('CabinType', 'cabin_type')
df = df.withColumnRenamed('DateFlown', 'date_flown')
df = df.withColumnRenamed('DatePub', 'date_pub')
df = df.withColumnRenamed('EntertainmentRating', 'entertainment_rating')
df = df.withColumnRenamed('FoodRating', 'food_rating')
df = df.withColumnRenamed('GroundServiceRating', 'ground_service_rating')
df = df.withColumnRenamed('OriginCountry', 'origin_country')
df = df.withColumnRenamed('OverallScore', 'overall_score')
df = df.withColumnRenamed('Recommended', 'recommended')
df = df.withColumnRenamed('Review', 'review')
df = df.withColumnRenamed('Route', 'route')
df = df.withColumnRenamed('SeatComfortRating', 'seat_comfort_rating')
df = df.withColumnRenamed('ServiceRating', 'service_rating')
df = df.withColumnRenamed('Slug', 'slug')
df = df.withColumnRenamed('Title', 'title')
df = df.withColumnRenamed('TravelType', 'travel_type')
df = df.withColumnRenamed('TripVerified', 'trip_verified')
df = df.withColumnRenamed('ValueRating', 'value_rating')
df = df.withColumnRenamed('WifiRating', 'wifi_rating')

In [6]:
df.printSchema()

root
 |-- aircraft: string (nullable = true)
 |-- airline_name: string (nullable = true)
 |-- cabin_type: string (nullable = true)
 |-- date_flown: string (nullable = true)
 |-- date_pub: string (nullable = true)
 |-- entertainment_rating: string (nullable = true)
 |-- food_rating: string (nullable = true)
 |-- ground_service_rating: string (nullable = true)
 |-- origin_country: string (nullable = true)
 |-- overall_score: string (nullable = true)
 |-- recommended: string (nullable = true)
 |-- review: string (nullable = true)
 |-- route: string (nullable = true)
 |-- seat_comfort_rating: string (nullable = true)
 |-- service_rating: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- title: string (nullable = true)
 |-- travel_type: string (nullable = true)
 |-- trip_verified: string (nullable = true)
 |-- value_rating: string (nullable = true)
 |-- wifi_rating: string (nullable = true)
 |-- unique_id: string (nullable = true)



In [7]:
df = df.where('overall_score in (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10) or overall_score is null')
df = df.withColumn('overall_score', col('overall_score').cast('integer'))
df = df.where('entertainment_rating in (0, 1, 2, 3, 4, 5) or entertainment_rating is null')
df = df.withColumn('entertainment_rating', col('entertainment_rating').cast('integer'))
df = df.where('food_rating in (0, 1, 2, 3, 4, 5) or food_rating is null')
df = df.withColumn('food_rating', col('food_rating').cast('integer'))
df = df.where('ground_service_rating in (0, 1, 2, 3, 4, 5) or ground_service_rating is null')
df = df.withColumn('ground_service_rating', col('ground_service_rating').cast('integer'))
df = df.where('seat_comfort_rating in (0, 1, 2, 3, 4, 5) or seat_comfort_rating is null')
df = df.withColumn('seat_comfort_rating', col('seat_comfort_rating').cast('integer'))
df = df.where('service_rating in (0, 1, 2, 3, 4, 5) or service_rating is null')
df = df.withColumn('service_rating', col('service_rating').cast('integer'))
df = df.where('value_rating in (0, 1, 2, 3, 4, 5) or value_rating is null')
df = df.withColumn('value_rating', col('value_rating').cast('integer'))
df = df.where('wifi_rating in (0, 1, 2, 3, 4, 5) or wifi_rating is null')
df = df.withColumn('wifi_rating', col('wifi_rating').cast('integer'))

df = df.where('cabin_type in ("Economy Class", "Business Class", "Premium Economy", "First Class") or cabin_type is null')

df = df.withColumn('trip_verified', when(col('trip_verified') == "NotVerified", "Not Verified")\
                   .otherwise(col('trip_verified')))
df = df.withColumn('trip_verified', when(col('trip_verified') == "Trip Verified,Trip Verified", "Trip Verified")\
                   .otherwise(col('trip_verified')))
df = df.withColumn('trip_verified', when(col('trip_verified') == "Not Verified,Not Verified", "Not Verified")\
                   .otherwise(col('trip_verified')))
df = df.withColumn('trip_verified', when(col('trip_verified') == "Not Verified,Not Verified", "Not Verified")\
                   .otherwise(col('trip_verified')))
df = df.withColumn('trip_verified', when(col('trip_verified') == "Chicago to Colorado Springs. Not Verified", "Not Verified")\
                   .otherwise(col('trip_verified')))
df = df.where('trip_verified in ("Trip Verified", "Not Verified") or trip_verified is null')

In [8]:
@pandas_udf('string', PandasUDFType.SCALAR)

def get_day_pub(v):
    return v.apply(lambda x: x.split(' ')[0])

@pandas_udf('string', PandasUDFType.SCALAR)

def get_month_pub(v):
    return v.apply(lambda x: x.split(' ')[1])

@pandas_udf('string', PandasUDFType.SCALAR)

def get_year_pub(v):
    return v.apply(lambda x: x.split(' ')[-1])

@pandas_udf('string', PandasUDFType.SCALAR)

def get_month_flown(v):
    return v.apply(lambda x: x.split(' ')[0])

@pandas_udf('string', PandasUDFType.SCALAR)

def get_year_flown(v):
    return v.apply(lambda x: x.split(' ')[1])

df = df.withColumn('date_pub', when(col('date_pub').isNull(), "None None None").otherwise(col('date_pub')))
df = df.withColumn('date_flown', when(col('date_flown').isNull(), "None None").otherwise(col('date_flown')))

df = df.withColumn('day_pub', get_day_pub(df.date_pub)).withColumn('day_pub', when(col('day_pub') == "None", None).otherwise(col('day_pub')))
df = df.withColumn('month_pub', get_month_pub(df.date_pub)).withColumn('month_pub', when(col('month_pub') == "None", None).otherwise(col('month_pub')))
df = df.withColumn('year_pub', get_year_pub(df.date_pub)).withColumn('year_pub', when(col('year_pub') == "None", None).otherwise(col('year_pub')))

df = df.withColumn('month_flown', get_month_flown(df.date_flown)).withColumn('month_flown', when(col('month_flown') == "None", None).otherwise(col('month_flown')))
df = df.withColumn('year_flown', get_year_flown(df.date_flown)).withColumn('year_flown', when(col('year_flown') == "None", None).otherwise(col('year_flown')))

df = df.withColumn('date_pub', when(col('date_pub') == "None None None", None).otherwise(col('date_pub')))
df = df.withColumn('date_flown', when(col('date_flown') == "None None", None).otherwise(col('date_flown')))

df = df.withColumn('day_pub_2', regexp_replace(col('day_pub'), 'st', ''))
df = df.withColumn('day_pub_2', regexp_replace(col('day_pub_2'), 'rd', ''))
df = df.withColumn('day_pub_2', regexp_replace(col('day_pub_2'), 'th', ''))
df = df.withColumn('day_pub_2', lpad(col('day_pub_2'), 2, '0'))

df = df.withColumn('month_pub_2', lit(None))
df = df.withColumn('month_pub_2', when(col('month_pub') == "January", "01").otherwise(col('month_pub_2')))
df = df.withColumn('month_pub_2', when(col('month_pub') == "February", "02").otherwise(col('month_pub_2')))
df = df.withColumn('month_pub_2', when(col('month_pub') == "March", "03").otherwise(col('month_pub_2')))
df = df.withColumn('month_pub_2', when(col('month_pub') == "April", "04").otherwise(col('month_pub_2')))
df = df.withColumn('month_pub_2', when(col('month_pub') == "May", "05").otherwise(col('month_pub_2')))
df = df.withColumn('month_pub_2', when(col('month_pub') == "June", "06").otherwise(col('month_pub_2')))
df = df.withColumn('month_pub_2', when(col('month_pub') == "July", "07").otherwise(col('month_pub_2')))
df = df.withColumn('month_pub_2', when(col('month_pub') == "August", "08").otherwise(col('month_pub_2')))
df = df.withColumn('month_pub_2', when(col('month_pub') == "September", "09").otherwise(col('month_pub_2')))
df = df.withColumn('month_pub_2', when(col('month_pub') == "October", "10").otherwise(col('month_pub_2')))
df = df.withColumn('month_pub_2', when(col('month_pub') == "November", "11").otherwise(col('month_pub_2')))
df = df.withColumn('month_pub_2', when(col('month_pub') == "December", "12").otherwise(col('month_pub_2')))


df = df.withColumn('month_flown_2', lit(None))
df = df.withColumn('month_flown_2', when(col('month_flown') == "January", "01").otherwise(col('month_flown_2')))
df = df.withColumn('month_flown_2', when(col('month_flown') == "February", "02").otherwise(col('month_flown_2')))
df = df.withColumn('month_flown_2', when(col('month_flown') == "March", "03").otherwise(col('month_flown_2')))
df = df.withColumn('month_flown_2', when(col('month_flown') == "April", "04").otherwise(col('month_flown_2')))
df = df.withColumn('month_flown_2', when(col('month_flown') == "May", "05").otherwise(col('month_flown_2')))
df = df.withColumn('month_flown_2', when(col('month_flown') == "June", "06").otherwise(col('month_flown_2')))
df = df.withColumn('month_flown_2', when(col('month_flown') == "July", "07").otherwise(col('month_flown_2')))
df = df.withColumn('month_flown_2', when(col('month_flown') == "August", "08").otherwise(col('month_flown_2')))
df = df.withColumn('month_flown_2', when(col('month_flown') == "September", "09").otherwise(col('month_flown_2')))
df = df.withColumn('month_flown_2', when(col('month_flown') == "October", "10").otherwise(col('month_flown_2')))
df = df.withColumn('month_flown_2', when(col('month_flown') == "November", "11").otherwise(col('month_flown_2')))
df = df.withColumn('month_flown_2', when(col('month_flown') == "December", "12").otherwise(col('month_flown_2')))

df = df.withColumn('date_pub_2', concat(col('month_pub_2'), lit('/'), col('day_pub_2'), lit('/'), col('year_pub')))
df = df.withColumn('date_flown_2', concat(col('month_flown_2'), lit('/'), lit('01'), lit('/'), col('year_flown')))

df = df.withColumn('date_pub_2', unix_timestamp(col('date_pub_2'), 'MM/dd/yyyy').cast('timestamp').cast('date'))
df = df.withColumn('date_flown_2', unix_timestamp(col('date_flown_2'), 'MM/dd/yyyy').cast('timestamp').cast('date'))

In [9]:
df = df.where('unique_id is not null')

In [10]:
df.count()

121296

In [12]:
df.write\
.mode('overwrite')\
.option('header', 'true')\
.csv("gs://msca-bdp-student-gcs/Group7_Final_Project/airline_reviews/airline_reviews_preprocessed.csv")

23/11/11 06:46:12 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [15]:
df.groupBy('aircraft').count().orderBy('count', ascending = False).toPandas()

,aircraft,count
0,None,88050
1,A320,4167
2,Boeing 737-800,2298
3,A330,1729
4,Boeing 787,1616
...,...,...
3359,A330-600neo,1
3360,A320/CRJ,1
3361,MD90 and CRJ200,1
3362,Bombardier CRJ 900,1


In [16]:
df.groupBy('airline_name').count().orderBy('count', ascending = False).toPandas()

,airline_name,count
0,American Airlines,4960
1,Spirit Airlines,4379
2,United Airlines,4082
3,British Airways,3231
4,Frontier Airlines,2836
...,...,...
540,Zambia Airways,1
541,Voyage Air,1
542,SVG Air,1
543,Nile Air,1


In [17]:
df.groupBy('cabin_type').count().orderBy('count', ascending = False).toPandas()

,cabin_type,count
0,Economy Class,96118
1,Business Class,15496
2,Premium Economy,4419
3,None,3017
4,First Class,2246


In [18]:
df.groupBy('date_flown_2').count().orderBy('count', ascending = False).toPandas()

,date_flown_2,count
0,None,38423
1,2015-08-01,1386
2,2019-08-01,1365
3,2019-04-01,1333
4,2018-07-01,1326
...,...,...
107,2014-07-01,11
108,2012-04-01,1
109,2014-05-01,1
110,2013-04-01,1


In [19]:
df.groupBy('date_pub_2').count().orderBy('count', ascending = False).toPandas()

,date_pub_2,count
0,None,3715
1,2015-01-19,297
2,2014-10-28,262
3,2015-01-14,256
4,2015-03-04,249
...,...,...
5035,2002-05-24,1
5036,2013-09-28,1
5037,2005-07-07,1
5038,2013-07-27,1


In [20]:
df.groupBy('origin_country').count().orderBy('count', ascending = False).toPandas()

,origin_country,count
0,UnitedStates,36311
1,UnitedKingdom,21264
2,Australia,11176
3,Canada,9760
4,Germany,3330
...,...,...
197,Mongolia,1
198,Syria,1
199,SaintLucia,1
200,Guinea,1


In [21]:
df.groupBy('entertainment_rating').count().orderBy('count', ascending = False).toPandas()

,entertainment_rating,count
0,0,50478
1,1,25052
2,4,13396
3,3,12491
4,5,11677
5,2,8202


In [22]:
df.groupBy('food_rating').count().orderBy('count', ascending = False).toPandas()

,food_rating,count
0,0,33821
1,1,27236
2,4,17336
3,3,15932
4,5,15426
5,2,11545


In [23]:
df.groupBy('ground_service_rating').count().orderBy('count', ascending = False).toPandas()

,ground_service_rating,count
0,0,41326
1,1,38047
2,5,14741
3,4,11495
4,3,8697
5,2,6990


In [24]:
df.groupBy('seat_comfort_rating').count().orderBy('count', ascending = False).toPandas()

,seat_comfort_rating,count
0,1,31161
1,4,23064
2,3,21298
3,5,16863
4,2,15091
5,0,13819


In [25]:
df.groupBy('recommended').count().orderBy('count', ascending = False).toPandas()

,recommended,count
0,no,71181
1,yes,50115


In [26]:
df.groupBy('route').count().orderBy('count', ascending = False).toPandas()

,route,count
0,None,38599
1,Sydney to Melbourne,139
2,Melbourne to Sydney,138
3,Vancouver to Toronto,99
4,Toronto to Vancouver,96
...,...,...
47507,MDT to LAS via ORD,1
47508,SIN to HGH,1
47509,Montreal to Amsterdam via Keflavik,1
47510,Doncaster to Cluj,1


In [27]:
df.groupBy('service_rating').count().orderBy('count', ascending = False).toPandas()

,service_rating,count
0,5,30090
1,1,29943
2,4,18843
3,3,15772
4,0,13920
5,2,12728


In [28]:
df.groupBy('slug').count().orderBy('count', ascending = False).toPandas()

,slug,count
0,american-airlines,4960
1,spirit-airlines,4379
2,united-airlines,4082
3,british-airways,3231
4,frontier-airlines,2836
...,...,...
541,nextjet,1
542,sudan-airways,1
543,nordstar,1
544,nordwind-airlines,1


In [29]:
df.groupBy('wifi_rating').count().orderBy('count', ascending = False).toPandas()

,wifi_rating,count
0,0,94367
1,1,16352
2,5,2996
3,3,2814
4,4,2434
5,2,2333


In [30]:
df.groupBy('value_rating').count().orderBy('count', ascending = False).toPandas()

,value_rating,count
0,1,45321
1,5,24952
2,4,21659
3,3,13715
4,2,13269
5,0,2380


In [31]:
df.groupBy('trip_verified').count().orderBy('count', ascending = False).toPandas()

,trip_verified,count
0,None,66239
1,Trip Verified,41950
2,Not Verified,13107


In [32]:
df.groupBy('travel_type').count().orderBy('count', ascending = False).toPandas()

,travel_type,count
0,None,38308
1,Solo Leisure,30407
2,Couple Leisure,21516
3,Family Leisure,18211
4,Business,12854


In [33]:
df.groupBy('overall_score').count().orderBy('count', ascending = False).toPandas()

,overall_score,count
0,1.0,39897
1,10.0,14214
2,9.0,12137
3,8.0,11359
4,2.0,11162
5,7.0,7284
6,3.0,7177
7,5.0,5090
8,4.0,4471
9,NaN,4331
